<a href="https://colab.research.google.com/github/rayarprasanth-004/AI-ML-WORKSHOP/blob/main/Copy_of_Dimensionality_Reduction_Skeleton_Code_Day_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Skeleton Code

In [1]:
import requests
from bs4 import BeautifulSoup
import zipfile
import io
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report
import numpy as np
import time

# Function to download and load dataset
def load_data():
    page_url = 'https://archive.ics.uci.edu/dataset/240/human+activity+recognition+using+smartphones'
    page_response = requests.get(page_url)
    if page_response.status_code == 200:
        soup = BeautifulSoup(page_response.content, 'html.parser')
        download_link = soup.select_one('a[href$=".zip"]')['href']
        full_download_url = 'https://archive.ics.uci.edu' + download_link
        response = requests.get(full_download_url)
        if response.status_code == 200:
            with zipfile.ZipFile(io.BytesIO(response.content)) as outer_zip:
                inner_zip_name = 'UCI HAR Dataset.zip'
                with outer_zip.open(inner_zip_name) as inner_zip_file:
                    with zipfile.ZipFile(io.BytesIO(inner_zip_file.read())) as inner_zip:
                        with inner_zip.open('UCI HAR Dataset/train/X_train.txt') as myfile:
                            df = pd.read_csv(myfile, delim_whitespace=True, header=None)
                        with inner_zip.open('UCI HAR Dataset/train/y_train.txt') as myfile_y:
                            y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)
    else:
        raise Exception("Failed to download or parse the dataset.")
    return df, y

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.cluster import KMeans
import numpy as np
import time

# Load dataset
df, y = load_data()

#TASK 1 - DO EDA and understand a little about the data.
#Only important thing is to know that it has a lot of features that don't make sense, just a
#bunch of readings from sensors.
#We think many of these features are redundant or irrelevant, and we want to find good features.


<ipython-input-1-836ccc4d8ebc>:30: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(myfile, delim_whitespace=True, header=None)
<ipython-input-1-836ccc4d8ebc>:32: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  y = pd.read_csv(myfile_y, delim_whitespace=True, header=None)


In [3]:
# Task 2: Encode class labels
# YOUR CODE HERE: Use LabelEncoder to encode class labels

label_encoder = LabelEncoder()
encoded_y = label_encoder.fit_transform(y.values.ravel())

In [4]:
# Task 3: Scale the features using StandardScaler
# YOUR CODE HERE: Apply StandardScaler to df

scaler = StandardScaler()
df_scaled = scaler.fit_transform(df)

In [5]:
# Task 4: Split the data into training and testing sets
# YOUR CODE HERE: Use train_test_split to split the data
X_train_full, X_test_full, y_train, y_test = train_test_split(df_scaled, encoded_y, test_size=0.2, random_state=42)

In [6]:
#TASK 5 - 1. Create a pipeline using Gaussian Naive Bayes
#         2. Fit the model to the training data
#         3. Predict values for test set
#         4. Print accuracy score
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# 1. Create the pipeline
classifier_pipeline = Pipeline([
    ('classifier', GaussianNB())
])

# 2. Fit the model
classifier_pipeline.fit(X_train_full, y_train)

# 3. Predict on the test set
y_pred = classifier_pipeline.predict(X_test_full)

# 4. Calculate and print the accuracy score
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7314751869476547


In [7]:
#TASK 6 - 1. Note the start time before defining the pipeline
#         2. Note the end time and report the difference as the time taken by the model training and inference.

import time

# 1. Note the start time
start_time = time.time()

# 2. Define the pipeline, fit the model, and predict
classifier_pipeline = Pipeline([
    ('classifier', GaussianNB())
])
classifier_pipeline.fit(X_train_full, y_train)
y_pred = classifier_pipeline.predict(X_test_full)

# 3. Note the end time and calculate the difference
end_time = time.time()
time_taken = end_time - start_time

# 4. Report the time taken
print(f"Time taken: {time_taken:.4f} seconds")


Time taken: 0.0571 seconds


In [8]:
# TASK 7 - K-Means for dimensionality reduction
n_clusters = 50
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
kmeans.fit(df_scaled.T)  # Transpose to treat features as data points
selected_features_indices = [np.random.choice(np.where(kmeans.labels_ == i)[0]) for i in range(n_clusters)]
selected_features = df_scaled[:, selected_features_indices]

In [9]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import accuracy_score

# Split the data with selected features into training and testing sets
X_train_selected, X_test_selected, y_train_selected, y_test_selected = train_test_split(
    selected_features, encoded_y, test_size=0.2, random_state=42
)

# Define and train the Gaussian Naive Bayes model
start_time = time.time()  # Record start time
gnb_selected = GaussianNB()
gnb_selected.fit(X_train_selected, y_train_selected)
end_time = time.time()  # Record end time

# Make predictions on the test set
y_pred_selected = gnb_selected.predict(X_test_selected)

# Calculate and report accuracy and time taken
selected_features_time = end_time - start_time
accuracy_selected = accuracy_score(y_test_selected, y_pred_selected)

print(f"Time taken with selected features: {selected_features_time:.4f} seconds")
print(f"Accuracy with selected features: {accuracy_selected:.4f}")

Time taken with selected features: 0.0113 seconds
Accuracy with selected features: 0.8572
